In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as plt
import numpy as np
#import cv2
from scipy.interpolate import interp1d

In [3]:
filename = 'IMG_1830.png'

In [21]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


img=mpimg.imread(filename)

def get_pt_onclick(event):
    global ix,iy, rgba
    ix, iy = event.xdata,event.ydata
    rgba = event.inaxes.get_images()[0].get_cursor_data(event).data
    print(ix, iy)
    
def get_pts_manual(event):
    fig.canvas.draw_idle()
    global ix,iy, pts_manual
    if event.key == 'm':
        plt.plot(ix,iy, 'x', color = 'tab:red') 
        pts_manual.append([ix,iy])


def cal_points(event): 
    fig.canvas.draw_idle()
    global ix,iy, calpts
    if len(calpts)<4:
        #print(event.key)
        if event.key == 'enter':
            calpts.append([ix,iy])
            plt.plot(ix,iy, 'rx')
#             if len(calpts)>=4:
#                 plt.close(plt.gcf())


def bound_points(event): 
    fig.canvas.draw_idle()
    global ix,iy, boundpts
    if len(boundpts)<4:
        #print(event.key)
        if event.key == 'b':
            boundpts.append([ix,iy])
            plt.plot(ix,iy, 'gx')

                
def line_color(event):
    global ix,iy,rgba,line_col
    if event.key == '-':
        print(event.key)
        line_col = rgba 
        
def mouse_eraser(event):
    fig.canvas.draw_idle()
    global ix,iy, pts
    if event.key == ' ':
        plt.plot(ix,iy, 'bo') 
        try:
            idx = np.where(((pts[0,:] - ix)**2 + (pts[1,:] - iy)**2)<300)[0]
            pts[:, idx] = 0
        except:
            print('bleh')
        

fig = plt.figure(figsize=(10,5))
cid = fig.canvas.mpl_connect('button_press_event', get_pt_onclick)
kid = fig.canvas.mpl_connect('key_press_event', cal_points)
bid = fig.canvas.mpl_connect('key_press_event', bound_points)
lid = fig.canvas.mpl_connect('key_press_event', line_color)
eid = fig.canvas.mpl_connect('key_press_event', mouse_eraser)
mid = fig.canvas.mpl_connect('key_press_event', get_pts_manual)

In [22]:
ix,iy = 0,0
line_col = [0,0,0,0]
calpts = []
boundpts = []
#pts = np.copy(mask2)
pts_manual = []
imgplot = plt.imshow(img)
plt.gca().plot([300,300], 'ro')
plt.show()

In [9]:
save_name = '5ps_18mW_250kHz'

1819.3457907159716 418.89461841070033
2235.747256321958 1481.6972527377977
3552.080616686094 1473.6215879502877
1926.8530781996992 1398.9216886658196
1916.7584972153118 619.6200366711012


In [10]:
calpts
# plt.gcf().canvas.mpl_connect('button_press_event', onclick)
# plt.gcf().canvas.mpl_connect('key_press_event', cal_points)

[[2235.747256321958, 1481.6972527377977],
 [3552.080616686094, 1473.6215879502877],
 [1926.8530781996992, 1398.9216886658196],
 [1916.7584972153118, 619.6200366711012]]

2108.555535918675 1449.3945935877575
3550.0617004892165 1433.2432640127374
2100.479871131165 1051.6681028028881
3483.4374659922587 1045.6113542122557


In [11]:
boundpts #[x, y]

[[2108.555535918675, 1449.3945935877575],
 [3550.0617004892165, 1433.2432640127374],
 [2100.479871131165, 1051.6681028028881],
 [3483.4374659922587, 1045.6113542122557]]

2488.1117809316465 1186.9354879936814
2584.7319271647198 1364.0673335567883
-


In [12]:
mask = np.sum((img - np.tile(line_col,(img.shape[0],img.shape[1],1)))**2, axis = 2)

In [13]:
plt.figure(2)
plt.imshow(mask)
plt.colorbar()

In [14]:
mask2 = np.copy(mask)
mask2[mask2>0.02] = 0
plt.figure(3)
plt.imshow(mask2, cmap = 'inferno_r')
plt.colorbar()
plt.show()

In [206]:
plt.figure(4)
plt.hist(mask2.flatten(), bins = 100)
plt.xlim(0,0.051)
plt.ylim(0,1e5)
plt.show()

In [15]:
X,Y=np.meshgrid(np.arange(mask2.shape[1]),np.arange(mask2.shape[0]))

In [16]:
bb = np.array(boundpts)
x0 = np.min(bb[:, 0]) #x
y0 = np.max(bb[:, 1]) #y
x1 = np.max(bb[:, 0]) #x
y1 = np.min(bb[:, 1]) #y
roi = np.zeros(mask2.shape)
roi[np.where((x0<X) & (y0>Y) & (x1>X) & (y1<Y))[0],np.where((x0<X) & (y0>Y) & (x1>X) & (y1<Y))[1]] = 1


In [409]:
# plt.figure(88)
# plt.imshow(roi*mask2)
# plt.show()

In [23]:
plt.colorbar()
plt.show()
plt.figure(1)
pts = np.array([np.where(mask2*roi>0)[1],np.where(mask2*roi>0)[0]])
plt.plot(pts[0,:], pts[1,:], 'rx')

1832.227272727272 392.2272727272725
2083.4095159386056 994.6607083825265
2587.3521835700944 1039.57638526869
2677.5249709064574 1056.399088964872
2755.0136044752444 1116.365893489064
3237.904188275319 1409.8876208969525


In [24]:
pts = np.hstack((pts,np.array(pts_manual).T))
pts = np.vstack((pts[0,:][(pts[0,:]>x0)&(pts[1,:]<y0)],pts[1,:][(pts[0,:]>x0)&(pts[1,:]<y0)]))
plt.figure(1)
plt.plot(pts[0,:], pts[1,:], 'gx')
plt.show()

In [ ]:
#Add in interpolation

In [43]:
calval = [-34,-24,0.01,0.1]
fx = interp1d([calpts[0][0],calpts[1][0]], [calval[0],calval[1]],bounds_error=None, fill_value = 'extrapolate')
fy = interp1d([calpts[2][1],calpts[3][1]],[calval[2],calval[3]],bounds_error=None, fill_value = 'extrapolate')
data = {}
data['x'] = fx(pts[0,:])
data['y'] = fy(pts[1,:])
np.save(save_name+'.npy',data)

5ps_18mW_250kHz.npy  IMG_1830.HEIC        README.md
Digitizer.ipynb      IMG_1830.png


In [40]:
plt.figure(48)
plt.plot(fx(pts[0,:]), fy(pts[1,:]), 'rx')

In [35]:
calpts[2][1]

1398.9216886658196

In [30]:
calpts

[[2235.747256321958, 1481.6972527377977],
 [3552.080616686094, 1473.6215879502877],
 [1926.8530781996992, 1398.9216886658196],
 [1916.7584972153118, 619.6200366711012]]

In [387]:
plt.figure(11)
plt.plot(pts[0,:], pts[1,:], 'o')
plt.show()

In [374]:
np.min(pts[0,:])

0.0

In [373]:
np.min(pts[1,:])

0.0

In [384]:
y0

1452.2495715517307

In [179]:
Y,X=np.meshgrid(np.arange(mask2.shape[0]),np.arange(mask2.shape[1]))

In [185]:
Y

array([[   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    1,    1,    1],
       [   2,    2,    2, ...,    2,    2,    2],
       ...,
       [4029, 4029, 4029, ..., 4029, 4029, 4029],
       [4030, 4030, 4030, ..., 4030, 4030, 4030],
       [4031, 4031, 4031, ..., 4031, 4031, 4031]])

In [102]:
line_col.data

array([0.52156866, 0.6117647 , 0.6901961 , 1.        ], dtype=float32)

In [210]:

plt.show()

In [5]:
# Coordinates that you want to Perspective Transform
pts1 = np.float32([[0,0],\
                   [0,1024],\
                   [1024,0],\
                   [1024, 1024]])


# Size of the Transformed Image
pts2 = np.float32([[0+xshift + dd2,0 + yshift],\
                   [0+xshift,1024 + yshift],\
                   [1024+xshift + dd,0 + yshift],\
                   [1024+xshift,1024 + yshift]])

MM = cv2.getPerspectiveTransform(pts1,pts2)

Digitizer.ipynb  IMG_1830.HEIC    IMG_1830.png     README.md


In [14]:
img.shape

(3024, 4032, 4)